<center><img src="https://storage.googleapis.com/arize-assets/fixtures/Arize-Phoenix-header.jpg" width="2000"/></center>


<center>
    <h2>LLM Application Tracing & Evaluation Workflows</h2>
    <h3>Exporting from Phoenix to Arize<br></h3>
</center>


This guide demonstrates how to use Arize for monitoring and debugging your LLM using Traces and Spans. We're going to use data from a chatbot built on top of Arize docs (https://docs.arize.com/arize/), with example query and retrieved text. Let's figure out how to understand how well our RAG system is working.

In this tutorial we will:
1. Build a RAG application using Llama-Index
1. Set up [Phoenix](https://docs.arize.com/phoenix) as a [trace collector](https://docs.arize.com/phoenix/tracing/llm-traces) for the Llama-Index application
2. Use Phoenix's [evals library](https://docs.arize.com/phoenix/evaluation/llm-evals) to compute LLM generated evaluations of our RAG app responses
3. Use arize SDK to export the traces and evaluations to Arize

You can read more about LLM tracing in Arize [here](https://docs.arize.com/arize/llm-large-language-models/llm-traces).

## Step 1: Install Dependencies 📚
Let's get the notebook setup with dependencies.

In [ ]:
# Dependencies needed to build the Llama Index RAG application
!pip install -qq gcsfs llama-index-llms-openai llama-index-embeddings-openai

# Dependencies needed to export spans and send them to our collector: Phoenix
!pip install -qq "llama-index-callbacks-arize-phoenix>=0.1.2"

# Install Phoenix to generate evaluations
!pip install -qq "arize-phoenix[evals]"

# Install Arize SDK with `Tracing` extra dependencies to export Phoenix data to Arize
!pip install -qq "arize[Tracing]>=7.12.0"

## Step 2: Set up Phoenix as a Trace Collector in our LLM app

To get started, launch the phoenix app. Make sure to open the app in your browser using the link below.

In [ ]:
import phoenix as px

session = px.launch_app()

Once you have started a Phoenix server, you can start your LlamaIndex application and configure it to send traces to Phoenix. To do this, you will have to add configure Phoenix as the global handler

In [ ]:
from llama_index.core import set_global_handler

set_global_handler("arize_phoenix")

That's it! The Llama-Index application we build next will send traces to Phoenix.

## Step 3: Build Your Llama Index RAG Application 📁

We start by setting your OpenAI API key if it is not already set as an environment variable.

In [ ]:
import os
from getpass import getpass

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

This example uses a `RetrieverQueryEngine` over a pre-built index of the Arize documentation, but you can use whatever LlamaIndex application you like. Download the pre-built index of the Arize docs from cloud storage and instantiate your storage context.

In [ ]:
from gcsfs import GCSFileSystem
from llama_index.core import StorageContext

file_system = GCSFileSystem(project="public-assets-275721")
index_path = "arize-phoenix-assets/datasets/unstructured/llm/llama-index/arize-docs/index/"
storage_context = StorageContext.from_defaults(
    fs=file_system,
    persist_dir=index_path,
)

We are now ready to instantiate our query engine that will perform retrieval-augmented generation (RAG). Query engine is a generic interface in LlamaIndex that allows you to ask question over your data. A query engine takes in a natural language query, and returns a rich response. It is built on top of Retrievers. You can compose multiple query engines to achieve more advanced capability.

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core import (
    Settings,
    load_index_from_storage,
)
from llama_index.embeddings.openai import OpenAIEmbedding


Settings.llm = OpenAI(model="gpt-4-turbo-preview")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
index = load_index_from_storage(
    storage_context,
)
query_engine = index.as_query_engine()

Let's test our app by asking a question about the Arize documentation:

In [ ]:
response = query_engine.query(
    "What is Arize and how can it help me as an AI Engineer?"
)
print(response)

Great! Our application works!

## Step 4: Use the instrumented Query Engine

We will download a dataset of questions for our RAG application to answer.

In [ ]:
from urllib.request import urlopen
import json

queries_url = "http://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/context-retrieval/arize_docs_queries.jsonl"
queries = []
with urlopen(queries_url) as response:
    for line in response:
        line = line.decode("utf-8").strip()
        data = json.loads(line)
        queries.append(data["query"])

queries[:5]

We use the instrumented query engine and get responses from our RAG app.

In [ ]:
from tqdm.notebook import tqdm

N = 10  # Sample size
qa_pairs = []
for query in tqdm(queries[:N]):
    resp = query_engine.query(query)
    qa_pairs.append((query, resp))

To see the questions and answers in phoenix, use the link described when we started the phoenix server

## Step 5: Run Evaluations on the data in Phoenix

We will use the phoenix client to extract data in the correct format for specific evaluations and the custom evaluators, also from phoenix, to run evaluations on our RAG application.

In [ ]:
from phoenix.session.evaluation import get_qa_with_reference

px_client = px.Client()  # Define phoenix client
queries_df = get_qa_with_reference(
    px_client
)  # Get question, answer and reference data from phoenix

Next, we enable concurrent evaluations for better performance.

In [ ]:
import nest_asyncio

nest_asyncio.apply()  # needed for concurrent evals in notebook environments

Then, we define our evaluators and run the evaluations

In [ ]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

eval_model = OpenAIModel(
    model="gpt-4-turbo-preview",
)
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)

Finally, we log the evaluations into Phoenix

In [ ]:
from phoenix.trace import SpanEvaluations

px_client.log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(
        eval_name="QA_Correctness", dataframe=qa_correctness_eval_df
    ),
)

## Step 6: Export data to Arize

### Step 6.a: Get data into dataframes

We extract the spans and evals dataframes from the phoenix client

In [ ]:
tds = px_client.get_trace_dataset()
spans_df = tds.get_spans_dataframe(include_evaluations=False)
spans_df.head()

In [ ]:
evals_df = tds.get_evals_dataframe()
evals_df.head()

### Step 6.b: Initialize arize client

In [ ]:
from arize.pandas.logger import Client

Sign up/ log in to your Arize account [here](https://app.arize.com/auth/login). Find your [space ID and API key](https://docs.arize.com/arize/api-reference/arize.pandas/client). Copy/paste into the cell below.

<img src="https://storage.googleapis.com/arize-assets/fixtures/copy-id-and-key.png" width="700">

In [ ]:
SPACE_ID = "SPACE_ID"  # Change this line
API_KEY = "API_KEY"  # Change this line

if SPACE_ID == "SPACE_ID" or API_KEY == "API_KEY":
    raise ValueError("❌ CHANGE SPACE_ID AND/OR API_KEY")
else:
    print(
        "✅ Import and Setup Arize Client Done! Now we can start using Arize!"
    )

arize_client = Client(
    space_id=SPACE_ID,
    api_key=API_KEY,
)
model_id = "tutorial-tracing-llama-index-rag-export-from-phoenix"
model_version = "1.0"

Lastly, we use `log_spans` from the arize client to log our spans data and, if we have evaluations, we can pass the optional `evals_dataframe`.

In [ ]:
response = arize_client.log_spans(
    dataframe=spans_df,
    evals_dataframe=evals_df,
    model_id=model_id,
    model_version=model_version,
)

# If successful, the server will return a status_code of 200
if response.status_code != 200:
    print(
        f"❌ logging failed with response code {response.status_code}, {response.text}"
    )
else:
    print(f"✅ You have successfully logged traces set to Arize")